In [ ]:
# 1. Instalar y cargar Kaggle, descomprimir el dataset
!pip install -q kaggle
from google.colab import files

# Subir kaggle.json con tus credenciales
uploaded = files.upload()

# Crear el directorio .kaggle y mover kaggle.json
!mkdir -p ~/.kaggle
!cp "{list(uploaded.keys())[0]}" ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

# Descargar y descomprimir el dataset CIFAR-10
!kaggle competitions download -c cifar-10
!unzip cifar-10.zip -d ./cifar-10

# Instalar librería para descomprimir .7z
!pip install py7zr
import py7zr

# Descomprimir las imágenes de entrenamiento
with py7zr.SevenZipFile('/content/cifar-10/train.7z', mode='r') as z:
    z.extractall(path='/content/cifar-10/train')

Saving kaggle.json to kaggle (1).json
cifar-10.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  cifar-10.zip
replace ./cifar-10/sampleSubmission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# 2. Hacer un muestreo del dataset para verificar su contenido y tamaño
import os
import random
from PIL import Image
import matplotlib.pyplot as plt

# Directorio de las imágenes
train_dir = '/content/cifar-10/train/train'

# Obtener una lista de archivos de imagen
image_files = [f for f in os.listdir(train_dir) if os.path.isfile(os.path.join(train_dir, f))]

# Seleccionar 5 imágenes aleatoriamente
random_images = random.sample(image_files, 5)

# Mostrar las imágenes y sus dimensiones
plt.figure(figsize=(10,5))
for i, image_file in enumerate(random_images):
    img = Image.open(os.path.join(train_dir, image_file))
    width, height = img.size
    print(f"Imagen: {image_file}, Tamaño: {width}x{height}")
    plt.subplot(1, 5, i + 1)
    plt.imshow(img)
    plt.axis('off')
    plt.title(image_file)
plt.show()

# **Clases de CIFAR-10**
airplane
automobile
bird
cat
deer
dog
frog
horse
ship
truck


In [ ]:
# 3. Tamaño de elementos para carpeta train
import os

# ruta de la carpeta
dir_path = r'/content/cifar-10/train/train'
count = 0
# Itera dentro de la carpeta, es un contador
for path in os.listdir(dir_path):
    # verifica si la ruta actual es un arhico
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('Número total de elementos:', count)

In [ ]:
# 4. Análisis rápido de las etiquetas de trainLabels.csv
import pandas as pd

# Cargar el CSV
labels_df = pd.read_csv('/content/cifar-10/trainLabels.csv')

labels_df.head()

In [ ]:
print(labels_df.size)

In [ ]:
# 3. Crear listas de imágenes y etiquetas
from sklearn.model_selection import train_test_split
import shutil
import os

# Asegurarse de que el ID esté correctamente formateado para coincidir con los nombres de archivo

images = labels_df['id'].apply(lambda x: f'{x}.png').values
labels = labels_df['label'].values

# Dividir en entrenamientos y validación
train_imgs, val_imgs, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, stratify=labels, random_state=42)

# Crear las carpetas de entrenamiento y validación
train_path = '/content/dataset/train'
val_path = '/content/dataset/val'

# Crear caroetas por clase dentro de val y train
for label in labels_df["label"].unique():
  os.makedirs(os.path.join(train_path, label), exist_ok=True)
  os.makedirs(os.path.join(val_path, label), exist_ok=True)

train_count = 0
val_count = 0

# Mover las imágenes de entrenamiento
for img, label in zip(train_imgs, train_labels):
    img_src = f'/content/cifar-10/train/train/{img}'
    img_dst = os.path.join(train_path, str(label), img)
    if os.path.exists(img_src):  # Verificar si la imagen existe
        shutil.move(img_src, img_dst)
        train_count += 1

print(f'Número de imágenes movidas a entrenamiento: {train_count}')

# Mover las imágenes de validación
for img, label in zip(val_imgs, val_labels):
    img_src = f'/content/cifar-10/train/train/{img}'
    img_dst = os.path.join(val_path, str(label), img)
    if os.path.exists(img_src):  # Verificar si la imagen existe
        shutil.move(img_src, img_dst)
        val_count += 1

print(f'Número de imágenes movidas a validación: {val_count}')

In [ ]:
# 3.1 Tamaño de elementos para carpeta train
import os

# ruta de la carpeta
dir_path = r'/content/dataset/train/airplane'
count = 0
# Itera dentro de la carpeta, es un contador
for path in os.listdir(dir_path):
    # verifica si la ruta actual es un arhico
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('Número total de elementos:', count)

In [ ]:
# 3.2 Tamaño de elementos para carpeta train
import os

# ruta de la carpeta
dir_path = r'/content/dataset/train/cat'
count = 0
# Itera dentro de la carpeta, es un contador
for path in os.listdir(dir_path):
    # verifica si la ruta actual es un arhico
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('Número total de elementos:', count)

In [ ]:
# 3.3 Tamaño de elementos para carpeta train
import os

# ruta de la carpeta
dir_path = r'/content/dataset/val/deer'
count = 0
# Itera dentro de la carpeta, es un contador
for path in os.listdir(dir_path):
    # verifica si la ruta actual es un arhico
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('Número total de elementos:', count)

In [ ]:
# 6. Verificar que las imágenes se han movido de forma exitosa a las carpetas
import os

def is_directory_empty(directory):
  return not os.listdir(directory)

directory_train =  "/content/dataset/train/airplane"

if is_directory_empty(directory_train):
  print("La carpeta está vacía para train.")
else:
  print("La carpeta no está vacía para train.")

directory_val =  "/content/dataset/val/airplane"

if is_directory_empty(directory_train):
  print("La carpeta está vacía para val.")
else:
  print("La carpeta no está vacía para val.")

In [ ]:
# 3.1. Tamaño de elementos para carpeta train
import os

# ruta de la carpeta
dir_path = r'/content/dataset/train/airplane'
count = 0
# Itera dentro de la carpeta, es un contador
for path in os.listdir(dir_path):
    # verifica si la ruta actual es un arhico
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('Número total de elementos:', count)

In [ ]:
# 3.2 Tamaño de elementos para carpeta train
import os

# ruta de la carpeta
dir_path = r'/content/dataset/train/automobile'
count = 0
# Itera dentro de la carpeta, es un contador
for path in os.listdir(dir_path):
    # verifica si la ruta actual es un arhico
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('Número total de elementos:', count)

In [ ]:
# 3.2 Tamaño de elementos para carpeta train
import os

# ruta de la carpeta
dir_path = r'/content/dataset/train/bird'
count = 0
# Itera dentro de la carpeta, es un contador
for path in os.listdir(dir_path):
    # verifica si la ruta actual es un arhico
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('Número total de elementos:', count)

In [ ]:
# 3.1. Tamaño de elementos para carpeta VAL
import os

# ruta de la carpeta
dir_path = r'/content/dataset/val/airplane'
count = 0
# Itera dentro de la carpeta, es un contador
for path in os.listdir(dir_path):
    # verifica si la ruta actual es un arhico
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('Número total de elementos:', count)

In [ ]:
### ENTRENAMIENTO DEL MODELO ####
# 7. Entrenar el modelo
!pip install ultralytics
from ultralytics import YOLO

# Cargar el modelo yolov8n-cls.pt para clasificación
model = YOLO('yolov8n-cls.pt')
# Entrenar el modelo con CIFAR-10
model.train(data='/content/dataset', epochs=20, imgsz=32)

In [ ]:
# 9. Predicción con imagen del dataset
import os
from PIL import Image

directory = "/content/dataset/train/airplane"

# Si la carpeta tiene imágenes, carga la primera imagen
image_path = os.path.join(directory, os.listdir(directory)[0])
image = Image.open(image_path)

# Realizar la predicción con el modelo YOLO
results = model.predict(image, conf=0.25, save=True)

# Mostrar la imagen con las predicciones
image.show()

In [ ]:
# 8. Visualizar los resultados de las gráficas YOLO
import os
from PIL import Image
import matplotlib.pyplot as plt

# Ruta del directorio donde se encuentran las imágenes
directory = "/content/runs/classify/train"

# Listar todos los archivos en el directorio
files = os.listdir(directory)
print("Estos son los archivos de resultado de entrenamiento", files)

# Filtrar solo los archivos de imagen (extensiones comunes)
image_files = [file for file in files if file.endswith(('.png', '.jpg', '.jpeg'))]

# Mostrar todas las imágenes
# Mostrar todas las imágenes
if image_files:
    for image_file in image_files:
        image_path = os.path.join(directory, image_file)

        # Cargar la imagen
        image = Image.open(image_path)

        # Mostrar la imagen
        plt.figure()
        plt.imshow(image)
        plt.axis('off')  # Ocultar los ejes
        plt.title(image_file)  # Título con el nombre del archivo
    plt.show()
else:
    print("No se encontraron imágenes en la carpeta.")

In [ ]:
!wget https://raw.githubusercontent.com/LuisAngelOlveraOlvera/YoloExamples/main/ImageTestYolo/rana.jpg

In [ ]:
# 11. Validar el modelo
from ultralytics import YOLO

# Cargar el modelo
model = YOLO("/content/runs/classify/train/weights/best.pt")

# Validar el modelo
metrics = model.val()

# Acceder a las métricas de clasificación
print(f"Top-1 Accuracy: {metrics.top1}")
print(f"Top-5 Accuracy: {metrics.top5}")
print(f"Fitness: {metrics.fitness}")

# Si quieres ver todas las métricas disponibles
print("\nTodas las métricas:")
for key, value in metrics.results_dict.items():
    print(f"{key}: {value}")

In [ ]:
# 10. Predicción con imagen de internet
import os
from PIL import Image

directory = "rana.jpg"

image = Image.open(directory)

# Realizar la predicción con el modelo YOLO
results = model.predict(image, conf=0.25, save=True)

# Mostrar la imagen con las predicciones
image.show()

In [ ]:
!wget https://raw.githubusercontent.com/LuisAngelOlveraOlvera/YoloExamples/main/ImageTestYolo/Pelusa.jpeg

In [ ]:
# 12 Realizar modelo desde el archivo .best

# Ruta del modelo YOLO
model_path = "/content/runs/classify/train/weights/best.pt"

# Cargar el modelo YOLO
model = YOLO(model_path)

# Ruta de la imagen local
image_path = "/content/Pelusa.jpeg"  # Cambia esto a la ruta real de tu imagen

# Cargar la imagen desde el sistema de archivos
image = Image.open(image_path)

# Verificar si la imagen está en RGB
if image.mode != 'RGB':
    image = image.convert('RGB')

# Redimensionar si es necesario (opcional)
image = image.resize((32, 32))  # Tamaño del CIFAR-10

# Realizar la predicción con el modelo YOLO
results = model(image)

# Procesar los resultados
probs = results[0].probs  # Obtener las probabilidades
clase_detectada = probs.top1  # Índice de la clase con mayor probabilidad
probabilidad = probs.top1conf.item()  # Probabilidad de la clase detectada

# Nombres de las clases del CIFAR-10
clases = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Mostrar la primera opción detectada
print(f"Primera opción detectada: {clases[clase_detectada]} con probabilidad {probabilidad:.2f}")

# Si quieres ver todas las probabilidades
for i, prob in enumerate(probs.data):
    print(f"{clases[i]}: {prob:.4f}")

In [ ]:
# 13. Mismo ejerecicio con un automóvil

!wget https://raw.githubusercontent.com/LuisAngelOlveraOlvera/YoloExamples/main/ImageTestYolo/cauto_test.jpg

# Ruta de la imagen local
image_path = "/content/cauto_test.jpg"  # Cambia esto a la ruta real de tu imagen

# Cargar la imagen desde el sistema de archivos
image = Image.open(image_path)

# Verificar si la imagen está en RGB
if image.mode != 'RGB':
    image = image.convert('RGB')

# Redimensionar si es necesario (opcional)
image = image.resize((32, 32))  # Tamaño del CIFAR-10

# Realizar la predicción con el modelo YOLO
results = model(image)

# Procesar los resultados
probs = results[0].probs  # Obtener las probabilidades
clase_detectada = probs.top1  # Índice de la clase con mayor probabilidad
probabilidad = probs.top1conf.item()  # Probabilidad de la clase detectada

# Nombres de las clases del CIFAR-10
clases = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Mostrar la primera opción detectada
print(f"Primera opción detectada: {clases[clase_detectada]} con probabilidad {probabilidad:.2f}")

# Si quieres ver todas las probabilidades
for i, prob in enumerate(probs.data):
    print(f"{clases[i]}: {prob:.4f}")

In [ ]:
# 14. Exportar modelo a onnx
from ultralytics import YOLO

# Cargar el modelo
model = YOLO("yolov8n-cls.pt")  # cargar un modelo official
model = YOLO("/content/runs/classify/train/weights/best.pt")  # load a custom trained model

# Export the model
model.export(format="onnx")

In [ ]:
!wget https://raw.githubusercontent.com/LuisAngelOlveraOlvera/YoloExamples/main/ImageTestYolo/gato3.JPG

In [ ]:
# 15. Inferencia con ONNX

import onnxruntime as ort
from PIL import Image
import numpy as np
import os

# Ruta del modelo ONNX exportado
onnx_model_path = "/content/runs/classify/train/weights/best.onnx"

# Cargar el modelo ONNX usando onnxruntime
session = ort.InferenceSession(onnx_model_path)

# Ruta de la imagen local
image_path = "/content/gato3.JPG"  # Cambia esto a la ruta real de tu imagen

# Cargar la imagen desde el sistema de archivos
image = Image.open(image_path)

# Verificar si la imagen está en RGB
if image.mode != 'RGB':
    image = image.convert('RGB')

# Redimensionar la imagen al tamaño adecuado para CIFAR-10 (32x32)
image = image.resize((32, 32))

# Convertir la imagen a un array numpy y escalar los valores de píxeles
image_np = np.array(image).astype(np.float32) / 255.0

# Cambiar el formato de la imagen a (1, 3, 32, 32) -> (batch_size, canales, alto, ancho)
image_np = np.transpose(image_np, (2, 0, 1))  # Cambiar los ejes para que esté en formato canales primero
image_np = np.expand_dims(image_np, axis=0)   # Añadir la dimensión de batch_size

# Realizar la predicción con el modelo ONNX
input_name = session.get_inputs()[0].name  # Obtener el nombre del primer input del modelo
output_name = session.get_outputs()[0].name  # Obtener el nombre del primer output del modelo
results = session.run([output_name], {input_name: image_np})

# Mostrar la primera opción detectada
predicciones = results[0][0]  # Acceder a las predicciones de la primera imagen
clase_detectada = np.argmax(predicciones)  # Obtener la clase con mayor probabilidad
probabilidad = np.max(predicciones)

# Nombres de las clases del CIFAR-10
clases = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

print(f"Primera opción detectada: {clases[clase_detectada]} con probabilidad {probabilidad:.2f}")



In [ ]:
# 16. Exportar modelo a TORCHSCRIPT
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n-cls.pt")
model = YOLO("/content/runs/classify/train/weights/best.pt")
model.export(format="torchscript")

In [ ]:
# 17 Inferencia con torchscript
import torch
from torchvision import transforms
from PIL import Image
import numpy as np

# Ruta del modelo TorchScript exportado
torchscript_model_path = "/content/runs/classify/train/weights/best.torchscript"

# Cargar el modelo TorchScript
model = torch.jit.load(torchscript_model_path)
model.eval()  # Poner el modelo en modo de evaluación

# Ruta de la imagen local
image_path = "/content/gato3.JPG"  # Cambia esto a la ruta real de tu imagen

# Cargar la imagen desde el sistema de archivos
image = Image.open(image_path)

# Verificar si la imagen está en RGB
if image.mode != 'RGB':
    image = image.convert('RGB')

# Definir las transformaciones
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Aplicar las transformaciones a la imagen
image_tensor = transform(image).unsqueeze(0)  # Añadir dimensión de batch

# Realizar la predicción con el modelo TorchScript
with torch.no_grad():
    output = model(image_tensor)

# Obtener las probabilidades y la clase predicha
probabilities = torch.nn.functional.softmax(output[0], dim=0)
clase_detectada = torch.argmax(probabilities).item()
probabilidad = probabilities[clase_detectada].item()

# Nombres de las clases del CIFAR-10
clases = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

print(f"Primera opción detectada: {clases[clase_detectada]} con probabilidad {probabilidad:.2f}")

In [ ]:
# 18 Inferencia con torchscript para otra clase
import torch
from torchvision import transforms
from PIL import Image
import numpy as np

# Ruta del modelo TorchScript exportado
torchscript_model_path = "/content/runs/classify/train/weights/best.torchscript"

# Cargar el modelo TorchScript
model = torch.jit.load(torchscript_model_path)
model.eval()  # Poner el modelo en modo de evaluación

# Ruta de la imagen local
image_path = "/content/cauto_test.jpg"  # Cambia esto a la ruta real de tu imagen

# Cargar la imagen desde el sistema de archivos
image = Image.open(image_path)

# Verificar si la imagen está en RGB
if image.mode != 'RGB':
    image = image.convert('RGB')

# Definir las transformaciones
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Aplicar las transformaciones a la imagen
image_tensor = transform(image).unsqueeze(0)  # Añadir dimensión de batch

# Realizar la predicción con el modelo TorchScript
with torch.no_grad():
    output = model(image_tensor)

# Obtener las probabilidades y la clase predicha
probabilities = torch.nn.functional.softmax(output[0], dim=0)
clase_detectada = torch.argmax(probabilities).item()
probabilidad = probabilities[clase_detectada].item()

# Nombres de las clases del CIFAR-10
clases = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

print(f"Primera opción detectada: {clases[clase_detectada]} con probabilidad {probabilidad:.2f}")